# NOMAD Cache Management Module

This notebook provides reusable cache management UI and logic for NOMAD API access. It can be imported into other dashboard notebooks.

## Usage

```python
# Import the cache management module
%run 'nomad_cache.ipynb'

# The following functions are now available:
# - create_cache_tab(): Creates and returns the cache management UI
```

## Requirements

- The `nomad_data.py` module should be available in the workspace for cache operations

In [ ]:
# Import required libraries
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, HTML
from IPython.display import display, clear_output
from datetime import datetime

# Import NOMAD data functionality
try:
    from nomad_data import get_cache_stats, clear_cache, CACHE_CONFIG
except ImportError:
    print("⚠️ Warning: nomad_data.py module not found. Cache management functionality will be limited.")

In [ ]:
def format_timestamp(ts_str):
    """Format timestamp string to readable format
    
    Args:
        ts_str (str): Timestamp string to format
        
    Returns:
        str: Formatted timestamp or 'N/A' if invalid
    """
    if not ts_str:
        return 'N/A'
    try:
        dt = datetime.fromisoformat(ts_str)
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except:
        return ts_str

In [ ]:
def create_cache_tab():
    """Create the cache management tab
    
    Returns:
        widgets.VBox: The cache management UI component
    """
    # Create widgets for cache management
    stats_output = widgets.Output()
    status_output = widgets.Output()

    # Create buttons for each cache type
    clear_buttons = {
        cache_type: widgets.Button(
            description=f'Clear {cache_type}',
            button_style='danger',
            layout={'width': '150px'}
        )
        for cache_type in CACHE_CONFIG.keys()
    }

    # Clear all button
    clear_all_button = widgets.Button(
        description='Clear All Cache',
        button_style='danger',
        icon='trash',
        layout={'width': '150px'}
    )

    # Refresh button
    refresh_button = widgets.Button(
        description='Refresh Stats',
        button_style='info',
        icon='sync',
        layout={'width': '150px'}
    )

    def update_stats():
        """Update the cache statistics display"""
        with stats_output:
            clear_output()
            stats = get_cache_stats()
            
            # Create a formatted table of stats
            print("Cache Statistics:")
            print("-" * 80)
            print(f"{'Cache Type':<15} {'Items':<8} {'Size (KB)':<12} {'Oldest':<25} {'Newest':<25}")
            print("-" * 80)
            
            for cache_type, cache_stats in stats.items():
                print(f"{cache_type:<15} {cache_stats['count']:<8} {cache_stats['size_kb']:<12.2f} ",
                      f"{format_timestamp(cache_stats['oldest']):<25} {format_timestamp(cache_stats['newest']):<25}")

    def on_clear_click(cache_type):
        def handler(b):
            with status_output:
                clear_output()
                print(f"Clearing {cache_type} cache...")
                clear_cache(cache_type)
                print(f"✓ {cache_type} cache cleared")
                update_stats()
        return handler

    def on_clear_all_click(b):
        with status_output:
            clear_output()
            print("Clearing all cache...")
            clear_cache()
            print("✓ All cache cleared")
            update_stats()

    def on_refresh_click(b):
        with status_output:
            clear_output()
            print("Refreshing cache statistics...")
            update_stats()
            print("✓ Statistics updated")

    # Set up button handlers
    for cache_type, button in clear_buttons.items():
        button.on_click(on_clear_click(cache_type))
    clear_all_button.on_click(on_clear_all_click)
    refresh_button.on_click(on_refresh_click)

    # Create the HTML for cache expiration times
    cache_expiry_html = "<ul>"
    for k, v in CACHE_CONFIG.items():
        cache_expiry_html += f"<li><b>{k}:</b> {v.get('expire_hours')} hours</li>"
    cache_expiry_html += "</ul>"

    # Create the cache management UI
    cache_ui = widgets.VBox([
        widgets.HTML("<h2>Cache Management</h2>"),
        widgets.HTML(f"<p>Cache expiration times:{cache_expiry_html}</p>"),
        widgets.HBox([refresh_button, clear_all_button]),
        widgets.HBox(list(clear_buttons.values())),
        status_output,
        stats_output
    ])

    # Initialize stats display
    update_stats()

    return cache_ui

## Example Usage

Here's an example of how to use this cache management module:

In [ ]:
# This cell demonstrates how to use the cache management module
# Not meant to be executed in this notebook directly

'''
# In your dashboard notebook:

# Import the cache management module
%run './nomad_cache.ipynb'

# Create the cache management tab
cache_tab = create_cache_tab()

# Display the tab
display(cache_tab)
'''